
<table class="ee-notebook-buttons" align="left">
    <td><a target="_parent"  href="https://github.com/MVOSlab-sdstate/AST426_Lab/blob/main/AST%20426L%20Lab%2003/AST426L%20Lab3.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_parent"  href="https://nbviewer.org/github/MVOSlab-sdstate/AST426_Lab/blob/main/AST%20426L%20Lab%2003/AST426L%20Lab3.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_parent"  href="https://colab.research.google.com/github/MVOSlab-sdstate/AST426_Lab/blob/main/AST%20426L%20Lab%2003/AST426L%20Lab3.ipynb"><img width=26px src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
    </table>

<p style="font-family: 'Georgia'; font-size: large;">Install the `sentinelhub` library. </p>

In [ ]:
!pip install sentinelhub

<p style="font-family: 'Georgia'; font-size: large;">Import required modules for Sentinel Hub and data visualization. </p>

In [ ]:
from sentinelhub import SHConfig, SentinelHubRequest, DataCollection, MimeType, CRS, BBox
import matplotlib.pyplot as plt
import numpy as np

<p style="font-family: 'Georgia'; font-size:X-large;">
  To get this, first log in to the dashboard of your Sentinel-hub account and follow the steps at the following link:
  <a href="https://docs.sentinel-hub.com/api/latest/api/overview/authentication/">Sentinel Hub</a>
</p>

<p style="font-family: 'Georgia'; font-size: large;">Configure Sentinel Hub with client credentials.</p>

In [ ]:
config = SHConfig()

# Replace with your actual client ID and secret from SentinelHub
config.sh_client_id = 'Your client ID'    #Your client id
config.sh_client_secret = 'Your Client Secret'    #your client secret

<p style="font-family: 'Georgia'; font-size: large;">Checks and warns if credentials are missing.</p>

In [ ]:
if not config.sh_client_id or not config.sh_client_secret:
    print("Warning: please provide the credentials (client ID and secret)!")

<p style="font-family: 'Georgia'; font-size: X-large;"> For this go, to maps.google.com and find a farm near Aurora, SD then click and choose two GPS coordinates in such a way that the first one is for top left corner of the field and the second one is bottom right corner of the field. The order of the GPS coordinates will be reversed when copid from maps.google.com. You should first put the longitude, latitude, longitude, latitude. </p>

<p style="font-family: 'Georgia'; font-size: large;">Defines the area of interest (AOI) using latitude and longitude.</p>

In [ ]:
# Define your area of interest (AOI)
bbox = BBox(bbox=[-96.687623,44.275802, -96.677689, 44.266921,], crs=CRS.WGS84)  # Example field in lat/long

<p style="font-family: 'Georgia'; font-size: large;">Requests and displays a true-color Sentinel-2 image.</p>

In [ ]:
from sentinelhub import bbox_to_dimensions

# Request a true color image from Sentinel-2
request_true_color = SentinelHubRequest(
    evalscript="""
    //VERSION=3
    function setup() {
      return {
        input: ["B04", "B03", "B02"],
        output: { bands: 3 }
      };
    }

    function evaluatePixel(sample) {
      return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02];
    }
    """,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C,
            time_interval=('2023-06-01', '2023-06-30')
        )
    ],
    responses=[
        SentinelHubRequest.output_response('default', MimeType.PNG)
    ],
    bbox=bbox,
    size=bbox_to_dimensions(bbox, resolution=10),  # 10m resolution
    config=config
)

# Fetch the true-color image
true_color_image = request_true_color.get_data()[0]

# Display the image
plt.imshow(true_color_image)
plt.title('True Color Image - Sentinel-2')
plt.axis('off')
plt.show()

<p style="font-family: 'Georgia'; font-size: large;">Requests, fetches, and displays an NDVI map from Sentinel-2.</p>

In [ ]:
request_ndvi = SentinelHubRequest(
    evalscript="""
    //VERSION=3
    function setup() {
      return {
        input: ["B08", "B04"],
        output: { bands: 1 }
      };
    }

    function evaluatePixel(sample) {
      let ndvi = (sample.B08 - sample.B04) / (sample.B08 + sample.B04);
      return [ndvi];
    }
    """,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C,
            time_interval=('2023-06-01', '2023-06-30')
        )
    ],
    responses=[
        SentinelHubRequest.output_response('default', MimeType.TIFF)
    ],
    bbox=bbox,
    size=bbox_to_dimensions(bbox, resolution=10),
    config=config
)

# Fetch NDVI data
ndvi_image = request_ndvi.get_data()[0]

# Display NDVI
plt.imshow(ndvi_image, cmap='RdYlGn')
plt.colorbar()
plt.title('NDVI Map for your chosen farm using the Sentinel-2 satellite data')
plt.axis('off')
plt.show()